In [3]:
# 🔹 Cell 1 — Imports & Config
import json
import torch
import torch.nn as nn
import sys
sys.path.append("..")

from torch.utils.data import Dataset, DataLoader
from models.sql_transformer import SQLTransformer
from src.utils import (
    tokens_to_ids,
    pad_sequence,
    create_attention_mask,
    set_seed,
    get_device
)
from src.vocab import PAD, TOKEN2ID

In [4]:
# =====================
# CONFIG
# =====================
PHASE1_CKPT = "checkpoints/phase1_model.pt"
PHASE2_CKPT = "checkpoints/phase2_model.pt"

DATASET_PATH = "../data/sql_ast/phase2_select_where.json"

EPOCHS = 20
BATCH_SIZE = 16
LR = 5e-4          # slightly lower than Phase-1
MAX_LEN = 12       # longer than Phase-1

In [5]:
# 🔹 Cell 2 — Phase-2 Dataset
#⚠️ Important
#Schema bindings are NOT used during training, only structure matters.
class Phase2Dataset(Dataset):
    def __init__(self, path):
        with open(path, "r") as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data[idx]["input_tokens"]

        ids = tokens_to_ids(tokens)
        ids = pad_sequence(ids, MAX_LEN, TOKEN2ID[PAD])
        mask = create_attention_mask(ids, TOKEN2ID[PAD])

        return (
            torch.tensor(ids, dtype=torch.long),
            torch.tensor(mask, dtype=torch.long)
        )

In [6]:
# 🔹 Cell 3 — Metrics (same as Phase-1)
def compute_prf(preds, labels, pad_id):
    tp = fp = fn = 0

    for p_seq, l_seq in zip(preds, labels):
        for p, l in zip(p_seq, l_seq):
            if l == pad_id:
                continue
            if p == l:
                tp += 1
            else:
                fp += 1
                fn += 1

    precision = tp / (tp + fp + 1e-9)
    recall = tp / (tp + fn + 1e-9)
    f1 = 2 * precision * recall / (precision + recall + 1e-9)

    return precision, recall, f1

In [7]:
# 🔹 Cell 4 — Load Phase-1 Model (CRITICAL STEP)
set_seed(42)
device = get_device()

dataset = Phase2Dataset(DATASET_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

model = SQLTransformer().to(device)

# 🔥 LOAD PHASE-1 WEIGHTS
model.load_state_dict(
    torch.load(PHASE1_CKPT, map_location=device)
)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss(ignore_index=TOKEN2ID[PAD])

print("✅ Phase-1 weights loaded, ready for Phase-2 training")

✅ Phase-1 weights loaded, ready for Phase-2 training


In [8]:
# 🔹 Cell 5 — Phase-2 Training Loop
best_f1 = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()

    total_loss = 0.0
    all_preds = []
    all_labels = []

    for input_ids, attention_mask in loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        optimizer.zero_grad()

        logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Teacher forcing
        logits = logits[:, :-1, :]
        labels = input_ids[:, 1:]

        loss = criterion(
            logits.reshape(-1, logits.size(-1)),
            labels.reshape(-1)
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

    precision, recall, f1 = compute_prf(
        all_preds,
        all_labels,
        pad_id=TOKEN2ID[PAD]
    )

    avg_loss = total_loss / len(loader)

    print(
        f"Epoch {epoch:02d} | "
        f"Loss: {avg_loss:.4f} | "
        f"P: {precision:.4f} | "
        f"R: {recall:.4f} | "
        f"F1: {f1:.4f}"
    )

    # Save best checkpoint
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), PHASE2_CKPT)
        print("💾 Phase-2 checkpoint saved")

Epoch 01 | Loss: 0.8833 | P: 0.8171 | R: 0.8171 | F1: 0.8171
💾 Phase-2 checkpoint saved
Epoch 02 | Loss: 0.2433 | P: 0.9355 | R: 0.9355 | F1: 0.9355
💾 Phase-2 checkpoint saved
Epoch 03 | Loss: 0.1226 | P: 0.9881 | R: 0.9881 | F1: 0.9881
💾 Phase-2 checkpoint saved
Epoch 04 | Loss: 0.0444 | P: 1.0000 | R: 1.0000 | F1: 1.0000
💾 Phase-2 checkpoint saved
Epoch 05 | Loss: 0.0231 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 06 | Loss: 0.0163 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 07 | Loss: 0.0128 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 08 | Loss: 0.0108 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 09 | Loss: 0.0095 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 10 | Loss: 0.0082 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 11 | Loss: 0.0072 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 12 | Loss: 0.0063 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 13 | Loss: 0.0058 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 14 | Loss: 0.0052 | P: 1.0000 | R: 1.0000 | F1: 1.0000
Epoch 15 | Loss: 0.0047 | P: 1.0000 | 

In [ ]:
# 🔹 Cell 6 — Load Phase-2 Model Later (Inference / Phase-3)
'''model = SQLTransformer().to(device)
model.load_state_dict(torch.load(PHASE2_CKPT, map_location=device))
model.eval()

print("✅ Phase-2 model loaded")'''